### Installation

In [28]:
#!pip install git+https://www.github.com/huggingface/transformers
#!pip install git+https://github.com/huggingface/accelerate
#!pip install bitsandbytes
#!pip install einops
#!pip install --upgrade torch torchvision
#!pip install scikit-learn
#!pip install matplotlib
#!pip install datasets
#!pip install Bio
#!pip install pybedtools
#!pip install tabulate

## Libraries

In [29]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification,BertForSequenceClassification, AutoModel, AutoConfig
from transformers.models.bert.configuration_bert import BertConfig
from datasets import load_dataset, Dataset

from sklearn import metrics 
from sklearn.model_selection import train_test_split

from Bio import SeqIO
from pybedtools import BedTool

import pandas as pd
import numpy as np
import scipy.stats as stats

import torch
from torch import nn

import importlib
import matplotlib.pyplot as plt
import seaborn as sns
import os

sns.set()

## Load Transformer Models

In [30]:
def load_models_and_tokenizers(models_names, bios_id, ft_model_type):
    models_tokenizers_dict = {}

    for model_name in models_names:
        model_ckpt = f"tanoManzo/{model_name}_ft_{bios_id}_{ft_model_type}"
        print(f"Loading model and tokenizer for: {model_ckpt}")

        try:
            # Load DNABERT model
            if 'dnabert2' in model_ckpt:
               model = BertForSequenceClassification.from_pretrained(model_ckpt, trust_remote_code=True)
               tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
            
            # Load Geneformer model
            elif 'Geneformer' in model_ckpt:
                tokenizer = AutoTokenizer.from_pretrained('tanoManzo/Geneformer_ft_Hepg2_1kbpHG19_DHSs_H3K27AC')
                model = AutoModelForSequenceClassification.from_pretrained(model_ckpt)

            # Load Gena models
            elif 'gena-' in model_ckpt:
                model = AutoModel.from_pretrained(model_ckpt, trust_remote_code=True)
                gena_module_name = model.__class__.__module__

                # BigBird model under Gena
                if 'bigbird' in model_ckpt:
                    cls = getattr(importlib.import_module(gena_module_name), 'BigBirdForSequenceClassification')
                else:
                    cls = getattr(importlib.import_module(gena_module_name), 'BertForSequenceClassification')
                
                model = cls.from_pretrained(model_ckpt, num_labels=2)
                tokenizer = AutoTokenizer.from_pretrained(model_ckpt, trust_remote_code=True)

            # Load generic model
            else:
                tokenizer = AutoTokenizer.from_pretrained(model_ckpt, trust_remote_code=True)
                model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, trust_remote_code=True)

            # Store the model and tokenizer in a dictionary
            models_tokenizers_dict[f"{model_name}_ft_{bios_id}"] = {'model': model, 'tokenizer': tokenizer}

        except Exception as e:
            print(f"Error loading {model_ckpt}: {str(e)}")

    return models_tokenizers_dict

# Example usage
#models_tokenizers_dict = load_models_and_tokenizers(models_names, bios_id, ft_model_type)

## Datasetes

### get fasta hg19/hg38 database

In [31]:
def get_chrom2seq(fasta_file, capitalize=True):

    chrom2seq = {}
    for seq in SeqIO.parse(fasta_file, "fasta"):
        chrom2seq[seq.description.split()[0]] = seq.seq.upper() if capitalize else seq.seq

    return chrom2seq
# Example usage
#chrom2seq = get_chrom2seq(FASTA_FILE_19)

### Get data

In [32]:
def data_preprocessing(type_data,name_data,dataset_path):
    
    updated_data_df = pd.DataFrame()
    path_file = f"{dataset_path}/{type_data}/{name_data}"

    if type_data == 'raQTL':
        old_data_df = pd.read_csv(path_file, sep='\t')
        updated_data_df['Chromosome'] = old_data_df['chr']
        updated_data_df['Position'] = old_data_df['SNPabspos']
        updated_data_df['Reference'] = old_data_df['ref']
        updated_data_df['Alternative'] = old_data_df['alt']
        updated_data_df['Value_Ratio'] = old_data_df['hepg2.alt.mean']/old_data_df['hepg2.ref.mean']
        updated_data_df['Value_Diff'] = old_data_df['hepg2.alt.mean']-old_data_df['hepg2.ref.mean']
        updated_data_df['Value_Pvalue_signed'] = -np.log10(old_data_df['hepg2.wilcox.p.value'])*np.sign(updated_data_df['Value_Diff'])
        updated_data_df['P_value'] = old_data_df['hepg2.wilcox.p.value']
    
    return updated_data_df

# Example usage
#data_df = data_preprocessing(type_data,name_data,dataset_path)

### Extract Sequences

In [33]:
def process_sequences(data_df, chrom2seq, length_bp=999):
    """
    Process sequences from a DataFrame and extract reference and alternative sequences.

    Parameters:
        mpra_df (pd.DataFrame): DataFrame containing chromosome, position, alt, and p-value columns.
        chrom2seq (dict): Dictionary mapping chromosomes to sequence data.
        length_bp (int): Length of the sequence to extract centered around each position.

    Returns:
        tuple: A tuple containing three lists:
            - seq_ref (list): List of reference sequences.
            - seq_alt (list): List of alternative sequences.
            - seq_val (list): List of values.
    """


    seq_ref = []
    seq_alt = []
    

    # Iterate over the DataFrame rows
    for idx, row in data_df.iterrows():
        chromosome = f"{row['Chromosome']}"
        abspos = row['Position']
        
        # Calculate the start and end positions for the sequence extraction
        start_pos = abspos - (length_bp // 2)-1
        end_pos = abspos + (length_bp // 2)  # Add 1 to ensure the length is exactly 1000 bp
        
        # Extract the sequence from the chromosome data
        seq = str(chrom2seq[chromosome][start_pos:end_pos])
        if len(seq) != length_bp:
            raise ValueError(f"Extracted sequence length {len(seq)} does not match the expected length {length_bp}.")
        
        half_len = len(seq) // 2

        #seq_ref.append(seq)
        seq_ref.append(f"{seq[:half_len]}{row['Reference']}{seq[half_len + 1:]}")
        

        # Create the alternative sequence by replacing the middle base with 'Alt'
        seq_alt.append(f"{seq[:half_len]}{row['Alternative']}{seq[half_len + 1:]}")

        if seq[half_len]!= row['Reference'] and seq[half_len]!= row['Alternative']:
            print("Warning Nucleaotide does NOT matched Ref or Alt")

    data_df['Seq_Reference'] = seq_ref
    data_df['Seq_Alternative'] = seq_alt
    return data_df

## Evaluation 

### Get model predictions

In [34]:
import torch

# Function to get predictions in batches
def get_predictions_raw(models_tokenizers_dict, seq_ref, seq_alt, device="cuda", batch_size=32):
    models_predictions = {}

    def tokenize_in_batches(sequence, tokenizer, max_length=512, batch_size=32):
        tokens = tokenizer(sequence, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
        for i in range(0, tokens['input_ids'].size(0), batch_size):
            yield {k: v[i:i+batch_size].to(device) for k, v in tokens.items()}

    for model_name, item in models_tokenizers_dict.items():
        model = item['model'].to(device).eval()
        tokenizer = item['tokenizer']

        print(f"Processing model: {model_name}")

        outputs_ref = []
        outputs_alt = []

        # Process in batches
        for inputs_ref in tokenize_in_batches(seq_ref, tokenizer, batch_size=batch_size):
            with torch.no_grad():
                batch_outputs_ref = model(**inputs_ref).logits.cpu()
                outputs_ref.append(batch_outputs_ref)
            torch.cuda.empty_cache()  # Clear memory after each batch

        for inputs_alt in tokenize_in_batches(seq_alt, tokenizer, batch_size=batch_size):
            with torch.no_grad():
                batch_outputs_alt = model(**inputs_alt).logits.cpu()
                outputs_alt.append(batch_outputs_alt)
            torch.cuda.empty_cache()  # Clear memory after each batch

        # Concatenate all batch results
        outputs_ref = torch.cat(outputs_ref, dim=0)
        outputs_alt = torch.cat(outputs_alt, dim=0)

        # Store results in CPU memory
        models_predictions[model_name] = {'ref': outputs_ref, 'alt': outputs_alt}

        # Free GPU memory by moving model to CPU and clearing cache
        model.to("cpu")
        torch.cuda.empty_cache()

    return models_predictions

# Usage
#models_predictions = get_predictions_raw(models_tokenizers_dict, data_df['Seq_Reference'].to_list(), data_df['Seq_Alternative'].to_list() , batch_size=8)

## Preprocessing Predictions

In [35]:
def compute_delta(outputs_ref_cpu, outputs_alt_cpu, seq_val):
    delta_ref = outputs_ref_cpu[:,1] - outputs_ref_cpu[:,0]
    delta_alt = outputs_alt_cpu[:,1] - outputs_alt_cpu[:,0]

    # Calculate the difference in logits between alternative and reference sequences
    log2_fold_change =  np.log2(torch.sigmoid(delta_alt)/torch.sigmoid(delta_ref))
    diff_alt_ref =  np.array(delta_alt)-np.array(delta_ref)

    # Compute the difference in the logit values for the positive class (enhancer)     
    log2_variant_expression_effect = np.log2(seq_val) 
            
    return np.array(log2_fold_change), log2_variant_expression_effect

## Linear Regression

In [36]:
def compute_regression_and_correlation(deltas):
    slope, intercept, r_val, p_val, std_err = stats.linregress(deltas)
    spearman_corr = stats.spearmanr(deltas[0], deltas[1]).correlation
    return slope, intercept, r_val, p_val, std_err, spearman_corr

## Main

### Parameters

In [37]:
# data path repo
dataset_path = "/data/Dcode/gaetano/repos/AI4Genomic/data"

# model name from huggingface.co/model name_id:model_name
models_names = [
 'dnabert2',
 #'nucleotide-transformer-v2-50m-multi-species',
 #'nucleotide-transformer-v2-100m-multi-species',
 #'nucleotide-transformer-v2-250m-multi-species',
 #'nucleotide-transformer-v2-500m-multi-species',
 #'nucleotide-transformer-500m-1000g',
 #'nucleotide-transformer-500m-human-ref',
 #'nucleotide-transformer-2.5b-1000g',
 #'nucleotide-transformer-2.5b-multi-species',
 #'Geneformer',
 #'gena-lm-bert-base-t2t',
 #'gena-lm-bert-large-t2t',
 #'gena-lm-bert-base-t2t-multi',
 #'gena-lm-bigbird-base-t2t',
 #'hyenadna-small-32k-seqlen-hf',
 #'hyenadna-medium-160k-seqlen-hf',
 #'hyenadna-medium-450k-seqlen-hf',
 #'hyenadna-large-1m-seqlen-hf'
 ]

# type of fine-tuned
ft_model_type = '1kbpHG19_DHSs_H3K27AC'

# samples for fine-tuning
#'BioS2'=Hela, 'BioS45'=neural progenitor cell, 'BioS73'=hepg2, 'BioS74'=k562
bios_ids = ['BioS2', 'BioS45', 'BioS73', 'BioS74']

FASTA_FILE_19 = "/data/Dcode/gaetano/repos/fasta_files/hg19.fa"
FASTA_FILE_38 = "/data/Dcode/gaetano/repos/fasta_files/hg38.fa"

bios_id = "BioS73"
models_tokenizers_dict = load_models_and_tokenizers(models_names, bios_id, ft_model_type)

chrom2seq = get_chrom2seq(FASTA_FILE_19)

dataset_path = "/data/Dcode/gaetano/repos/AI4Genomic/data"
type_data = 'raQTL'
name_data = 'hepg2.sign.id.LP190708.txt'

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading model and tokenizer for: tanoManzo/dnabert2_ft_BioS73_1kbpHG19_DHSs_H3K27AC


model.safetensors:   0%|          | 0.00/357M [00:00<?, ?B/s]

### Run

In [38]:
data_df = data_preprocessing(type_data, name_data, dataset_path)
data_df = process_sequences(data_df, chrom2seq)
data_df = data_df.iloc[:100]

models_predictions = get_predictions_raw(models_tokenizers_dict, data_df['Seq_Reference'].to_list(), data_df['Seq_Alternative'].to_list() , batch_size=8)

for model_name in models_predictions.keys():
    out_ref = models_predictions[model_name]['ref']
    out_alt = models_predictions[model_name]['alt']

    log2_prediction, log2_exp = compute_delta(out_ref, out_alt, data_df['Value_Ratio'].to_list())
    models_predictions[model_name]['log2_prediction'] = log2_prediction

    print(compute_regression_and_correlation((log2_prediction, log2_exp)))

Processing model: dnabert2_ft_BioS73
(13.140624943004761, 0.9880731812586995, 0.40957110265799423, 2.3225334722565703e-05, 2.9566571764360727, 0.422106210621062)


In [39]:
models_predictions

{'dnabert2_ft_BioS73': {'ref': tensor([[ 0.0387, -0.2584],
          [ 1.0463, -1.4867],
          [ 0.5962, -0.9592],
          [ 0.4781, -0.8132],
          [ 0.5145, -0.8579],
          [ 1.1292, -1.5772],
          [ 0.0583, -0.2877],
          [ 0.2440, -0.5298],
          [-0.3360,  0.2572],
          [ 0.4126, -0.7133],
          [-0.8249,  0.9355],
          [ 1.2763, -1.7423],
          [ 1.3494, -1.8104],
          [-1.2690,  1.4577],
          [-1.0124,  1.1699],
          [ 1.2426, -1.7074],
          [ 1.3365, -1.7981],
          [-0.3579,  0.3120],
          [-0.9870,  1.1271],
          [-0.9789,  1.1178],
          [-0.7929,  0.8970],
          [-1.2418,  1.4284],
          [-0.8016,  0.8989],
          [ 0.0971, -0.3225],
          [-0.3315,  0.2520],
          [-0.8031,  0.9029],
          [-0.6470,  0.6817],
          [-0.8435,  0.9565],
          [-0.5263,  0.5278],
          [-0.3140,  0.2487],
          [-0.2650,  0.1573],
          [-1.2956,  1.4912],
          [

In [40]:
from scipy import stats
from typing import Callable, Tuple, Dict, Any

def compute_regression_and_correlation(deltas):
    slope, intercept, r_val, p_val, std_err = stats.linregress(deltas)
    spearman_corr = stats.spearmanr(deltas[0], deltas[1]).correlation
    return slope, intercept, r_val, p_val, std_err, spearman_corr

def get_linregress_stats(model, model_predictions, seq_val, significance=True, filter_significant=filter_significant):
    if model in ['trednet', 'Enformer', 'Sei', 'borzoi']:
        deltas = compute_delta_trednet_sei_enformer_borzoi(model_predictions, seq_val)
    else:
        outputs_ref_cpu, outputs_alt_cpu = model_predictions.values()
        deltas = compute_delta(outputs_ref_cpu, outputs_alt_cpu, seq_val)

    if significance:
        deltas =  np.array(deltas[0])[filter_significant],  np.array(deltas[1])[filter_significant]
    
    return compute_regression_and_correlation(deltas), deltas

NameError: name 'filter_significant' is not defined

## TredNet & Sei & Enformer & Borzoi

In [ ]:
models_to_add = ['trednet', 'Sei', 'Enformer', 'borzoi']
models_names += [model for model in models_to_add if model not in models_names]
models_names

## Load predictions 

In [ ]:
import pickle
import numpy as np
import pandas as pd

# Load TredNet predictions
with open("/data/Dcode/gaetano/repos/Transformers4Genomic/data/mpra/trednet_predictions_GSE68331.pkl", 'rb') as f:
    data = pickle.load(f, encoding='latin1')
    outputs_ref_cpu_tred = np.squeeze(data['predictions_ref'])
    outputs_alt_cpu_tred = np.squeeze(data['predictions_alt'])
    models_predictions['trednet'] = outputs_alt_cpu_tred/outputs_ref_cpu_tred


path_sei_pred = "/data/Dcode/gaetano/CNNplusModels/sei-framework/sei_mpra_GSE68331_prediction_diff_full.pkl"
with open(path_sei_pred, 'rb') as f:
    data = pickle.load(f, encoding='latin1')
    delta_sei_out = np.squeeze(data['diff_ratio'])
models_predictions['Sei'] = delta_sei_out


# Load Enformer predictions
with open("/data/Dcode/gaetano/CNNplusModels/enformer-pytorch/enformer_mpra_GSE68331_prediction_diff_full.pkl", 'rb') as f:
    data = pickle.load(f, encoding='latin1')
    delta_enformer_out = np.squeeze(data['diff_ratio'])
models_predictions['Enformer'] = delta_enformer_out


# Load borzoi predictions
with open("/data/Dcode/gaetano/CNNplusModels/baskerville/borzoi/examples/borzoi_models_sample/results/borzoi_mpra_GSE68_prediction_diff_full.pkl", 'rb') as f:
    data = pickle.load(f, encoding='latin1')
    delta_borzoi_out = np.squeeze(data['diff_ratio'])
models_predictions['borzoi'] = delta_borzoi_out


## Delta Computation

In [ ]:
def compute_delta(outputs_ref_cpu, outputs_alt_cpu, seq_val):
    delta_ref = outputs_ref_cpu[:,0] - outputs_ref_cpu[:,1]
    delta_alt = outputs_alt_cpu[:,0] - outputs_alt_cpu[:,1]

    # Calculate the difference in logits between alternative and reference sequences
    fold_change =  np.array(delta_alt)/np.array(delta_ref)
    #fold_change =  np.array(delta_alt)-np.array(delta_ref)

    # Compute the difference in the logit values for the positive class (enhancer)     
    log2_variant_expression_effect = np.log2(fold_change) 
    #log2_variant_expression_effect = fold_change

    # Compute the difference in expression between alt and ref
    log2_exp = []
    log2_variant_expression_filter = []
    for s_m, s_v in zip(log2_variant_expression_effect,seq_val):
        if not np.isnan(s_m):
            log2_exp.append(s_v)
            log2_variant_expression_filter.append(s_m)
        else:
            log2_exp.append(s_v)
            log2_variant_expression_filter.append(0)
            
    return log2_exp, log2_variant_expression_filter

In [ ]:
def compute_delta_trednet_sei_enformer_borzoi(outputs_cpu, seq_val):
    # Calculate the difference in logits between alternative and reference sequences
    fold_change =  outputs_cpu

    # Compute the difference in the logit values for the positive class (enhancer)
    log2_variant_expression_effect = np.log2(fold_change) 
    #log2_variant_expression_effect = fold_change 
    
    # Compute the difference in expression between alt and ref
    log2_exp = []
    log2_variant_expression_filter = []
    for s_m, s_v in zip(log2_variant_expression_effect,seq_val):
        if not np.isnan(s_m):
            log2_exp.append(s_v)
            log2_variant_expression_filter.append(s_m)
            
    return log2_exp, log2_variant_expression_filter

## Plots

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List, Tuple, Any, Dict

def plot_regression(df_significant, df_all, model, r_val_significant, p_val_significant, r_val_all, p_val_all):
    plt.figure(figsize=(10, 6))
    
    # Plot non-significant data (all data)
    sns.regplot(
        x='Model Output Difference',
        y='Expression Difference',
        data=df_all,
        scatter_kws={'alpha': 0.1, 'color': 'blue'},
        line_kws={'color': 'blue', 'label': f'All Data: r={r_val_all:.3f}, p={p_val_all:.3f}'}
    )
    
    # Plot significant data
    sns.regplot(
        x='Model Output Difference',
        y='Expression Difference',
        data=df_significant,
        scatter_kws={'alpha': 0.5, 'color': 'red'},
        line_kws={'color': 'red', 'label': f'Significant Data: r={r_val_significant:.3f}, p={p_val_significant:.3f}'}
    )
    
    plt.ylabel('Expression Value log2(Alternative/Reference)')
    plt.xlabel('Model Output Value log2(Alternative/Reference)')
    plt.title(f'{model}')
    plt.legend()
    plt.show()

def process_models(models_names, models_predictions, seq_val):
    for model in models_names:
        # Fetch the model statistics and deltas for significant data
        (slope_significant, intercept_significant, r_val_significant, p_val_significant, std_err_significant, spearman_corr_significant), (delta_exp_significant, delta_model_out_significant) = get_linregress_stats(model, models_predictions[model], seq_val, True, filter_significant)
        
        # Fetch the model statistics and deltas for all data (non-significant included)
        (slope_all, intercept_all, r_val_all, p_val_all, std_err_all, spearman_corr_all), (delta_exp_all, delta_model_out_all) = get_linregress_stats(model, models_predictions[model], seq_val, False, filter_significant)
        
        # Prepare data for plotting
        df_significant = pd.DataFrame({
            'Expression Difference': delta_exp_significant,
            'Model Output Difference': delta_model_out_significant
        })
        
        df_all = pd.DataFrame({
            'Expression Difference': delta_exp_all,
            'Model Output Difference': delta_model_out_all
        })

        # Plot the regression with overlapping data
        plot_regression(df_significant, df_all, model, r_val_significant, p_val_significant, r_val_all, p_val_all)
        print('done')

# Call the process_models function with your parameters
process_models(models_names, models_predictions, np.log2(seq_val))
